In [30]:
import cx_Oracle
import os
import io

In [31]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [32]:
cur = connection.cursor()

In [33]:
cur.execute('DELETE FROM "C##PPAWLUK"."DOKUMENT"')
cur.execute('commit')

In [34]:
nameDocuments = os.listdir("dokumenty")

In [35]:
def clearTxt(myfile):
    myfile = myfile.replace('\n', ' ').replace(',', '').replace('.', '').replace('!', '').replace('?', '').replace('/',
                                                                                                                   '').replace(
        '\\', '').replace(';', '').replace('"', ' ').replace('\'', ' ').replace(']', '').replace('[', '').replace(')',
                                                                                                                  '').replace(
        '(',
        '').replace(
        ':', '').replace('`', '').replace('@', '').replace('#', '').replace('$', '').replace('%', '').replace(
        '^', '').replace('&', '').replace('*', '').replace('-', ' ').replace('_', ' ').replace('+', ' ').replace('=',
                                                                                                                 ' ').replace(
        '0', '').replace('1', '').replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6',
                                                                                                              '').replace(
        '7', '').replace('8', '').replace('9', '').replace('>', ' ').replace('<', ' ').replace('{', '').replace('}',
                                                                                                                '').replace(
        '§', '').replace('\', '').replace('\\ufeff', '').replace(' ', ' ').replace('–', ' ').replace('“', '').replace(
        '„', '').replace('…', '').replace('	', '').replace('’', ' ').replace('”', '').replace('  ', ' ').replace(
        '  ', ' ')
    return myfile

In [36]:
def stopList(myfile):
    replaceFile = myfile
    for word in myfile:
        print(word)
        stopword = cur.execute(
            'select STWORD from STOPWORDS where STWORD = \'{}\' and rownum=1'.format(
                word))
        for i in stopword:
            replaceFile.remove(i[0])
    removeTab = {'	', '\n', ''}
    for remove in removeTab:
        while (remove in replaceFile):
            replaceFile.remove(remove)
    return replaceFile

In [37]:
def slowikPolskiego(replaceFile):
    j = 0
    for word in replaceFile:
        dictionary = cur.execute(
            'select slow_forma_podstawowa from slownik where slow_wyraz = \'{}\' and rownum=1'.format(
                word))
        for i in dictionary:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile


In [38]:
def slowikSynonimow1(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select slow_forma_podstawowa from slownik_synonimow where slow_wyraz = \'{}\' and slow_wyraz != slow_forma_podstawowa and rownum=1 '.format(
                word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [39]:
def slowikSynonimow2(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select slowo_podstawowe from synonimy_orginalne where (synonim1 = \'{}\' or synonim2 = \'{}\' or synonim3 = \'{}\' or synonim4 = \'{}\' or synonim5 = \'{}\' or synonim6 = \'{}\' or synonim7 = \'{}\' or synonim8 = \'{}\' or synonim9 = \'{}\' or synonim10 = \'{}\' or synonim11 = \'{}\' or synonim12 = \'{}\' or synonim13 = \'{}\' or synonim14 = \'{}\' or synonim15 = \'{}\' or synonim16 = \'{}\' or synonim17 = \'{}\' or synonim18 = \'{}\') and NOT EXISTS(select slowo_podstawowe from synonimy_orginalne where slowo_podstawowe = \'{}\') and rownum=1 '.format(
                word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [ ]:
def slowikSynonimow3(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select COLUMN1 from synonimy2 where (synonim1 = \'{}\' or synonim2 = \'{}\' or synonim3 = \'{}\' or synonim4 = \'{}\' or synonim5 = \'{}\' or synonim6 = \'{}\' or synonim7 = \'{}\' or synonim8 = \'{}\' or synonim9 = \'{}\' or synonim10 = \'{}\' or synonim11 = \'{}\' or synonim12 = \'{}\' or synonim13 = \'{}\' or synonim14 = \'{}\' or synonim15 = \'{}\' or synonim16 = \'{}\' or synonim17 = \'{}\' or synonim18 = \'{}\') and NOT EXISTS(select slowo_podstawowe from synonimy_orginalne where slowo_podstawowe = \'{}\') and rownum=1 '.format(
                word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word,word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [40]:
def pdfparser(data, nr):
    file = io.open(data, mode="r", encoding='utf-8-sig')
    myfile = file.read()
    myfile = myfile.lower()
    myfile = clearTxt(myfile)
    myfile = myfile.split(' ')
    replaceFile = slowikPolskiego(myfile)
    replaceFile = stopList(replaceFile)
    replaceFile = slowikSynonimow2(replaceFile)
    replaceFile = stopList(replaceFile)
    txt = ''
    for i in range(0, len(replaceFile), 1):
        txt = txt + ' ' + (replaceFile.__getitem__(i))
        if i % 200 == 0 and i != 0:
            cur.execute(
                'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                    nameDocuments[nr], txt))
            cur.execute('commit')
            txt = ''
    if (txt != ''):
        cur.execute(
            'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                nameDocuments[nr], txt))
        cur.execute('commit')

In [41]:
for i in range(0, len(nameDocuments), 1):
    pdfparser("D:\BSpy\!materiały na studia\Praca inżynierska\Implementacja\PorownywanieDokumentow\dokumenty\{}".format(
        nameDocuments[i]), i)

dziennik
ustawa
rzeczpospolita
polski
warszawa
dzień
październik
rok
pozycja
wyrok
trybunał
konstytucyjny
z
październik
rok
akt
trybunał
konstytucyjny
w
przewodniczyć
tuleja
sprawozdawca
po
w
artykuł
usta
punkt
ustawa
z
listopad
rok
o
i
postępowanie
przed
konstytucyjny
dziennik
u
na
niejawny
w
październik
rok
skarga
konstytucyjny
o
zgodność
artykuł
ustawa
z
czerwiec
rok
kodeks
postępowanie
karny
dziennik
u
pozycja
z
w
w
wyłączać
dopuszczalność
zaskarżyć
zażalenie
postanowienie
wydać
na
artykuł
kodeks
postępowanie
karny
i
się
oskarżyciel
posiłkowy
o
mowa
w
kodeks
postępowanie
karny
z
w
z
usta
artykuł
usta
artykuł
usta
i
konstytucja
rzeczpospolita
polski
orzekać
artykuł
ustawa
z
czerwiec
rok
kodeks
postępowanie
karny
dziennik
u
pozycja
z
w
obowiązujący
do
rok
to jest
do
poprzedzać
wejście
w
artykuł
punkt
ustawa
z
wrzesień
rok
o
ustawa
kodeks
postępowanie
karny
oraz
inny
dziennik
u
w
wyłączać
dopuszczalność
zaskarżyć
zażalenie
postanowienie
wydać
na
artykuł
ustawa
kodeks
postępowanie
karn

lokować
dwanaście miesięcy
desygnacja
dwanaście miesięcy
chorągiew
powód
plaster
chorągiew
addendum
akt prawny
powód
plaster
chorągiew
mąż
górnolotny
być w stanie
nadzór
chorągiew
członek rządu
casus
będący w
casus
będący w
lokalizacja
akt prawny
boss
lekkiego serca
członek rządu
spośród
doba
dwanaście miesięcy
casus
określony
niwa
członek rządu
casus
będący w
akt prawny
odbierać
buława
chorągiew
przyrządzić
spośród
kaliber
barwa
buława
metaliczny
połączenie
buława
być w stanie
ćwiek
złocisty
barwa
buława
metaliczny
metaliczny
buława
przyrządzić
barwa
złocisty
addendum
akt prawny
odbierać
addendum
aktualny
akt prawny
addendum
akt prawny
odbierać
addendum
aktualny
akt prawny
chorągiew
nadzór
aktualny
być w stanie
bawić się
chorągiew
akt prawny
luzować
aktualny
akt prawny
akt prawny
napływać
los
dziura
doba
doba
członek rządu
casus
będący w
miast
akt prawny
addendum
akt prawny
członek rządu
casus
będący w
spośród
członek rządu
dwanaście miesięcy
akt prawny
casus
będący w
spośród
doba
dwa

w zastępstwie
tom
dziennik
ustawa
pozycja
załącznik
do
minister
obrona
narodowy
z
wrzesień
rok
pozycja
załącznik
numer
ramowy
program
szkolenie
dla
morski
kategoria
a
ogólny
a
statystyk
i
obserwacja
bar
geometria
i
liniowy
bar
rachunek
różniczkowy
i
różniczkowy
bar
rachunek
prawdopodobieństwo
i
bar
bar
informatyk
i
komunikacja
bar
system
komputerowy
bar
oprogramowanie
biurowy
bar
element
programować
bar
usługa
sieciowy
i
bar
baza
dana
bar
fizyk
bar
kinematyka
bar
grawimetria
bar
magnetyzm
bar
teoria
fala
bar
fala
elektromagnetyczny
bar
optyk
geometryczny
bar
laser
bar
przetwornik
i
czas
bar
d
bar
środek
i
oznakowanie
nawigacyjny
bar
światowy
morski
system
łączność
alarmowy
i
bar
nawigacyjny
mapa
morski
bar
publikacja
nautyczny
bar
kompas
bar
procedura
bezpieczeństwo
na
bar
zasada
bezpieczeństwo
praca
i
praktyk
morski
bar
lin
i
bar
holować
i
przyrząd
pomiarowy
bar
kotwiczyć
bar
cumować
bar
e
bar
podstawa
meteorologia
i
meteorologiczny
bar
wiatr
i
morski
bar
prognozować
pogoda
bar
dzienn

popularność
popularność
albo
ćwiczenia
popularność
adekwatny
akt prawny
napływać
los
dziura
doba
doba
członek rządu
nacjonalistyczny
miast
egzemplarz książki
akt prawny
addendum
akt prawny
członek rządu
nacjonalistyczny
spośród
doba
dwanaście miesięcy
addendum
makietowy
plan
ćwiczenia
obiekt
hipoteza
linearny
kalkulacja
kalkulacja
możliwość
organizm
aplikacja
kodować
posługa
charyzma
hipoteza
bałwan
bałwan
konwerter
miara
organizm
globalny
organizm
ogłoszenie
postępowanie
bezdno
artykuł wiary
fabrykacja
pragmatyk
delator
ciągnąć
aparat
bałwan
oczekiwać
aura
akt prawny
obiekt
pierwotny
kalkulacja
organizm
miara
formować
hipoteza
badania
czworokąt foremny
hydrologiczny
mąż
bezdno
obiekt
organizm
aparat
globalny
organizm
organizm
bierny
organizm
linia
łatwość
organizm
organizm
organizm
kołtuński
pakiet
organizm
organizm
postępowanie
miara
fabrykacja
postępowanie
fabrykacja
artretyzm
gadka szmatka
ciek
gadka szmatka
gadka szmatka
miara
gadka szmatka
ciek
makieta
porozumienie
nurt
edytować


niniejszy
dyplom
posiadać
wiedza
umiejętność
doświadczenie
wykonywać
praca
hydrograficzny
stosownie
wymaganie
określić
publikacja
sekunda
miejsce
zdjąć
kategoria
imię
nazwisko
podpis
posiadacz
dyplom
szef
biuro
hydrograficzny
marynarka
wojenny
dzień
dziennik
ustawa
pozycja
załącznik
numer
wzór
dyplom
hydrograf
morski
kategoria
bar
dyplom
hydrograf
morski
numer
biuro
hydrograficzny
marynarka
wojenny
okaziciel
niniejszy
dyplom
posiadać
wiedza
umiejętność
doświadczenie
wykonywać
praca
hydrograficzny
stosownie
wymaganie
określić
publikacja
sekunda
bar
for
bar
miejsce
zdjąć
kategoria
bar
imię
nazwisko
podpis
posiadacz
dyplom
szef
biuro
hydrograficzny
marynarka
wojenny
dzień
wykonywaniu
prac
hydrograficznych
stosownie
wymagań
określonych
publikacji
iho
miejsce
zdjęcie
kategoria
a
nazwisko
mp
podpis
posiadacza
dyplomu
szef
biura
hydrograficznego
marynarki
wojennej
gdynia
dnia
dziennik
ustaw
poz
załącznik
nr
wzór
dyplomu
hydrografa
morskiego
kategorii
dyplom
hydrografa
morskiego
numer
biuro
hy

In [42]:
cur.close()

In [43]:
connection.close()